In [1]:
!pip install torch torchvision torchaudio torchtext
!pip install pytorch-forecasting
!pip install mlxtend
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 846.0/846.0 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 62.6 MB/s eta 0:00:00


In [2]:
import torch.utils.data as data

# DeepGA is an algorithm for evolve a convolutional neural network.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
"""
carpeta con los .py
"""
%cd  /content/drive/MyDrive/Workspace/Actividad_DeepGA/DeepGA_kvasir

/content/drive/MyDrive/Workspace/Actividad_DeepGA/DeepGA_kvasir


In [5]:
from DeepGA.Operators import *
from DeepGA.EncodingClass import Encoding
from DeepGA.Decoding import *
from DeepGA.DataReader import *
from DeepGA.DistributedTraining import *
from DeepGA.DeepGA import *

In [6]:
%pwd

'/content/drive/MyDrive/Workspace/Actividad_DeepGA/DeepGA_kvasir'

In [7]:
!nvidia-smi


Wed Jan 14 16:17:26 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [8]:
import os
import shutil

src_dir = "/content/drive/MyDrive/Workspace/Actividad_DeepGA/DeepGA_kvasir/Radiografias_dxs_pulpares"
dst_dir = "/content/radiografias_dxs_pulpares"

def ignore_junk(current_dir, entries):
    ignored = []
    for e in entries:
        if e.startswith(".") or e.upper() == "__MACOSX":
            ignored.append(e)
    return set(ignored)

if os.path.exists(dst_dir):
    shutil.rmtree(dst_dir)

shutil.copytree(src_dir, dst_dir, ignore=ignore_junk)
print("Copied clean dataset to:", dst_dir)


Copied clean dataset to: /content/radiografias_dxs_pulpares


In [9]:
data_dir = "/content/radiografias_dxs_pulpares"

# Data reader

In [ ]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

def load_radiographs_folder_dataset(
    data_dir,
    image_size=128,
    batch_size=32,
    val_split=0.1,
    test_split=0.1,
    seed=42,
    num_workers=2,
):
    # X-rays are usually grayscale; force 1 channel for consistency
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5]),
    ])

    full_dataset = datasets.ImageFolder(root=data_dir, transform=transform)

    total = len(full_dataset)
    test_size = int(total * test_split)
    val_size = int(total * val_split)
    train_size = total - val_size - test_size

    if train_size <= 0 or val_size <= 0 or test_size <= 0:
        raise ValueError(
            f"Invalid split sizes. total={total}, train={train_size}, val={val_size}, test={test_size}"
        )

    generator = torch.Generator().manual_seed(seed)
    train_set, val_set, test_set = random_split(
        full_dataset, [train_size, val_size, test_size], generator=generator
    )
    pin = torch.cuda.is_available()
    train_dl = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin)
    val_dl = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin)
    test_dl = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin)

    n_channels = 1
    n_classes = len(full_dataset.classes)
    out_size = image_size

    print("Classes:", full_dataset.classes)
    print("Counts:", total, "train/val/test =", train_size, val_size, test_size)

    return train_dl, val_dl, test_dl, n_channels, n_classes, out_size


# Estratificado

In [10]:
import numpy as np
import torch
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from sklearn.model_selection import StratifiedShuffleSplit
from collections import Counter

def make_stratified_loaders_v2(
    data_dir,
    image_size=128,
    batch_size=32,
    val_split=0.15,
    test_split=0.15,
    seed=42,
    num_workers=2,
):
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5]),
    ])

    ds = datasets.ImageFolder(root=data_dir, transform=transform)
    y = np.array(ds.targets)
    idx = np.arange(len(ds))

    # 1) Split: (train+val) vs test
    sss_test = StratifiedShuffleSplit(n_splits=1, test_size=test_split, random_state=seed)
    trainval_idx, test_idx = next(sss_test.split(idx, y))

    # 2) Split: train vs val (sobre trainval)
    y_trainval = y[trainval_idx]
    val_rel = val_split / (1.0 - test_split)  # val como fracción de lo que quedó
    sss_val = StratifiedShuffleSplit(n_splits=1, test_size=val_rel, random_state=seed)
    train_rel, val_rel_idx = next(sss_val.split(trainval_idx, y_trainval))

    train_idx = trainval_idx[train_rel]
    val_idx = trainval_idx[val_rel_idx]

    pin = torch.cuda.is_available()
    train_dl = DataLoader(Subset(ds, train_idx), batch_size=batch_size, shuffle=True,  num_workers=num_workers, pin_memory=pin)
    val_dl   = DataLoader(Subset(ds, val_idx),   batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin)
    test_dl  = DataLoader(Subset(ds, test_idx),  batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=pin)

    print("Counts:", len(train_idx), len(val_idx), len(test_idx))
    return train_dl, val_dl, test_dl, 1, len(ds.classes), image_size, ds, train_idx, val_idx, test_idx


In [11]:
from collections import Counter
import numpy as np

def show_split_distribution(ds, split_idx, name):
    y = np.array(ds.targets)[split_idx]
    c = Counter(y)
    total = len(split_idx)
    print(f"\n{name} total = {total}")
    for class_id, class_name in enumerate(ds.classes):
        n = c.get(class_id, 0)
        print(f"  {class_name:35s} {n:4d}  ({n/total:.2%})")

train_dl, val_dl, test_dl, n_channels, n_classes, out_size, ds, train_idx, val_idx, test_idx = make_stratified_loaders_v2(
    data_dir="/content/radiografias_dxs_pulpares",
    image_size=128,
    batch_size=32,
    val_split=0.15,
    test_split=0.15,
    num_workers=2,
)

show_split_distribution(ds, train_idx, "TRAIN")
show_split_distribution(ds, val_idx,   "VAL")
show_split_distribution(ds, test_idx,  "TEST")


Counts: 256 55 55

TRAIN total = 256
  Necrosis Pulpar                       70  (27.34%)
  Previamente iniciado                  17  (6.64%)
  Previamente tratado                   30  (11.72%)
  Pulpa Normal                          44  (17.19%)
  Pulpitis irreversible asintomática   39  (15.23%)
  Pulpitis irreversible sintomática    53  (20.70%)
  Pulpitis reversible                    3  (1.17%)

VAL total = 55
  Necrosis Pulpar                       15  (27.27%)
  Previamente iniciado                   4  (7.27%)
  Previamente tratado                    6  (10.91%)
  Pulpa Normal                           9  (16.36%)
  Pulpitis irreversible asintomática    9  (16.36%)
  Pulpitis irreversible sintomática    11  (20.00%)
  Pulpitis reversible                    1  (1.82%)

TEST total = 55
  Necrosis Pulpar                       15  (27.27%)
  Previamente iniciado                   4  (7.27%)
  Previamente tratado                    7  (12.73%)
  Pulpa Normal                    

### Call DeepGA for evolve CNN´s

In [12]:
'''Defining DeepGA hyperparameters'''
#Convolutional layers
FSIZES = [3, 5, 7, 9] # Odd Sizes Are Preferred
NFILTERS = [8, 16, 32, 64, 128, 256]

#Pooling layers
PSIZES = [2,3] #[2,3,4,5]
PTYPE = ['max', 'avg']

#Fully connected layers
NEURONS = [16, 32, 64, 128, 256] # for big layers

In [13]:
import DeepGA.Operators as ops

ops.FSIZES = FSIZES
ops.NFILTERS = NFILTERS
ops.PSIZES = PSIZES
ops.NEURONS = NEURONS


In [14]:
EXECUTION_ID =708

In [16]:
'''Defining DeepGA hyperparameters'''

#Defining learning rate
lr = 1e-4

#Maximun and minimum numbers of layers to initialize networks
min_conv = 2 # 30
max_conv = 30 # 60
min_full = 1
max_full = 6 # 10
max_params = 9e6
train_epochs = 15 # Epochs to train the best individual found by the GA

'''Genetic Algorithm Parameters'''
cr = 0.7   # Crossover rate
mr = 0.5   # Mutation rate
N = 25      # Population size 20 Se mantuvo en 20
T = 35 #30      # Number of generations
t_size = 5 # tournament size 5
w = 0.1    # penalization weight   0.3

chck_dir = 'point/'  # Root folder for dumping/loading pickle
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == "cuda":
    print(f"✅ Using GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version (torch): {torch.version.cuda}")
else:
    print("⚠️ Using CPU (no CUDA/GPU detected)")


train_dl, val_dl, test_dl, n_channels, n_classes, out_size, ds, train_idx, val_idx, test_idx = make_stratified_loaders_v2(
    data_dir=data_dir,
    image_size=128,
    batch_size=32,
    val_split=0.15,
    test_split=0.15,
    seed=42,
    num_workers=2,
)

# opcional ->>> verificar distribución
show_split_distribution(ds, train_idx, "TRAIN")
show_split_distribution(ds, val_idx,   "VAL")
show_split_distribution(ds, test_idx,  "TEST")

loss_func = nn.CrossEntropyLoss()
execution_ID = EXECUTION_ID #Execution ID for checkpoint, change it for a new execution



✅ Using GPU: Tesla T4
CUDA version (torch): 12.6
Counts: 256 55 55

TRAIN total = 256
  Necrosis Pulpar                       70  (27.34%)
  Previamente iniciado                  17  (6.64%)
  Previamente tratado                   30  (11.72%)
  Pulpa Normal                          44  (17.19%)
  Pulpitis irreversible asintomática   39  (15.23%)
  Pulpitis irreversible sintomática    53  (20.70%)
  Pulpitis reversible                    3  (1.17%)

VAL total = 55
  Necrosis Pulpar                       15  (27.27%)
  Previamente iniciado                   4  (7.27%)
  Previamente tratado                    6  (10.91%)
  Pulpa Normal                           9  (16.36%)
  Pulpitis irreversible asintomática    9  (16.36%)
  Pulpitis irreversible sintomática    11  (20.00%)
  Pulpitis reversible                    1  (1.82%)

TEST total = 55
  Necrosis Pulpar                       15  (27.27%)
  Previamente iniciado                   4  (7.27%)
  Previamente tratado                 

In [18]:

results, pop, bestind  = deepGA(execution_ID, True, train_epochs = train_epochs, train_dl=train_dl, val_dl=val_dl,  lr=lr,
                       min_conv=min_conv, max_conv=max_conv, min_full=min_full, max_full=max_full, max_params=max_params,
                       cr=cr, mr=mr, N=N, T=T, t_size=t_size, w=w, device=device, chck_dir=chck_dir,
                       n_channels =  n_channels , n_classes=n_classes, out_size = out_size, loss_func=loss_func)

# training to more epochs

finalEpochs = 150#100
CNNModel = final_evaluation(execution_ID, bestind, train_dl, val_dl, lr, max_params, w, device, finalEpochs, loss_func, chck_dir, n_channels =  n_channels , n_classes=n_classes, out_size = out_size)

Re-Initialize population
--------------------------------------------
Generation:  33
0.5894834939393939 0.5636363636363636 1601031
0.5906827828282828 0.5636363636363636 1493095
0.4506753808080808 0.41818181818181815 2311943
0.4506753808080808 0.41818181818181815 2311943
0.4269527141414141 0.41818181818181815 4446983
0.5572094575757576 0.5454545454545454 3032967
0.5894834939393939 0.5636363636363636 1601031
0.4506753808080808 0.41818181818181815 2311943
0.5894834939393939 0.5636363636363636 1601031
0.5894834939393939 0.5636363636363636 1601031
0.5622308717171717 0.5636363636363636 4053767
0.5894834939393939 0.5636363636363636 1601031
Best fitness:  0.5906827828282828
Best accuracy:  0.5636363636363636
Best No. of Params:  1493095
No. of Conv. Layers:  7
No. of FC Layers:  1
--------------------------------------------
Generation:  34
0.46218746969696967 0.41818181818181815 1275855
0.33057464545454546 0.2727272727272727 1339191
0.45352150606060604 0.43636363636363634 3528519
0.590682782

In [ ]:
import os
import pandas as pd
import pickle


### save the result in memeoruy in txt file


In [19]:
"""
save the result in memeoruy in txt file

"""
def save_deepga_run_summary_txt(results, pop, bestind, out_path):
    os.makedirs(os.path.dirname(out_path), exist_ok=True)

    lines = []
    lines.append("=== DeepGA Run Summary ===\n")

    # ---- results (DataFrame) ----
    lines.append("== results (per generation) ==\n")
    if isinstance(results, pd.DataFrame):
        lines.append(results.to_string(index=True))
    else:
        lines.append(str(results))
    lines.append("\n\n")

    # ---- bestind ----
    lines.append("== bestind (best individual) ==\n")
    try:
        enc = bestind[0]
        lines.append(f"fitness: {bestind[1]}\n")
        lines.append(f"accuracy: {bestind[2]}\n")
        lines.append(f"params: {bestind[3]}\n")
        lines.append("\n-- encoding details --\n")
        # Try to print useful attributes if present
        for attr in ["n_conv", "n_full", "first_level", "second_level"]:
            if hasattr(enc, attr):
                lines.append(f"{attr}: {getattr(enc, attr)}\n")
            else:
                lines.append(f"{attr}: <not present>\n")
    except Exception as e:
        lines.append(f"Could not format bestind: {e}\n")
        lines.append(str(bestind) + "\n")
    lines.append("\n")

    # ---- pop (top-k) ----
    lines.append("== pop (final population) ==\n")
    lines.append(f"population_size: {len(pop)}\n\n")

    # Sort by fitness descending and dump top K
    top_k = min(10, len(pop))
    pop_sorted = sorted(pop, key=lambda x: x[1], reverse=True)

    lines.append(f"-- top {top_k} individuals by fitness --\n")
    for i in range(top_k):
        ind = pop_sorted[i]
        enc = ind[0]
        lines.append(f"\n[{i+1}] fitness={ind[1]}  acc={ind[2]}  params={ind[3]}\n")
        for attr in ["n_conv", "n_full"]:
            if hasattr(enc, attr):
                lines.append(f"  {attr}: {getattr(enc, attr)}\n")

    with open(out_path, "w", encoding="utf-8") as f:
        f.write("".join(lines))

    print("Saved:", out_path)

# Example usage (choose a path)
out_txt = "/content/drive/MyDrive/Workspace/Actividad_DeepGA/DeepGA_kvasir/deepga_summary_708.txt"
save_deepga_run_summary_txt(results, pop, bestind, out_txt)


Saved: /content/drive/MyDrive/Workspace/Actividad_DeepGA/DeepGA_kvasir/deepga_summary_708.txt


### Trying to only train the model

In [20]:
chck_dir='/content/drive/MyDrive/Workspace/Actividad_DeepGA/DeepGA_kvasir/point'



In [ ]:

ga_ckpt = os.path.join(chck_dir, f"{EXECUTION_ID}_checkpoint.pkl")

with open(ga_ckpt, "rb") as f:
    state = pickle.load(f)

pop = state["pop"]  # list of [encoding, fitness, accuracy, params]

# Choose criterion:
best_by_fitness = max(pop, key=lambda x: x[1])   # same as DeepGA leader
best_by_accuracy = max(pop, key=lambda x: x[2])  # if you prefer pure val accuracy

bestind = best_by_fitness  # or best_by_accuracy

print("Loaded bestind:",
      "fitness=", bestind[1],
      "acc=", bestind[2],
      "params=", bestind[3],
      "generation_saved=", state["t"])


Loaded bestind: fitness= 0.45208866000000003 acc= 0.4166666666666667 params= 1145567 generation_saved= 30


In [ ]:
finalEpochs = 150

CNNModel = final_evaluation(
    EXECUTION_ID, bestind,
    train_dl, val_dl,
    lr, max_params, w, device,
    finalEpochs, loss_func, chck_dir,
    n_channels=n_channels, n_classes=n_classes, out_size=out_size,
)


Training final model from best individual
0.47708865999999994 0.4444444444444444 1145567
Execution time:  1006.332914563  seconds
Execution time:  0.27953692071194447  hours
Accuracy:  0.4444444444444444


In [ ]:
state_path = os.path.join(chck_dir, f"Model_Exec_{execution_ID}_Epoch_{finalEpochs}.pt")
torch.save(CNNModel.state_dict(), state_path)
print("✅ Saved state_dict:", state_path)


bestind_path = os.path.join(chck_dir, f"bestind_exec_{execution_ID}.pkl")
with open(bestind_path, "wb") as f:
    pickle.dump(bestind, f)
print("✅ Saved bestind:", bestind_path)

✅ Saved state_dict: point/Model_Exec_707_Epoch_150.pt
✅ Saved bestind: point/bestind_exec_707.pkl


# Cómo cargarlo luego (necesitas reconstruir la arquitectura)

In [ ]:
state_path = os.path.join(chck_dir, f"Model_Exec_{execution_ID}_Epoch_{finalEpochs}.pt")

# Rebuild architecture from bestind
network = decoding(bestind[0], n_channels, out_size, n_classes)
model = CNN(bestind[0], network[0], network[1], network[2]).to(device)

# Load weights
model.load_state_dict(torch.load(state_path, map_location=device))
model.eval()

print("✅ Loaded model with state_dict")

# Carger bestind

In [ ]:
with open(bestind_path, "rb") as f:
    bestind = pickle.load(f)

# TEST

In [ ]:
def evaluate_model(model, data_loader, device, loss_fn=None):
    model.eval()
    total = 0
    correct = 0
    total_loss = 0.0

    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            outputs = model(images)
            preds = outputs.argmax(dim=1)

            total += labels.size(0)
            correct += (preds == labels).sum().item()

            if loss_fn is not None:
                loss = loss_fn(outputs, labels)
                total_loss += loss.item() * labels.size(0)

    acc = correct / total if total else 0.0
    avg_loss = (total_loss / total) if (loss_fn is not None and total) else None
    return acc, avg_loss

# ejemplo de uso:
test_acc, test_loss = evaluate_model(CNNModel, test_dl, device, loss_fn=loss_func)
print("Test accuracy:", test_acc)
if test_loss is not None:
    print("Test loss:", test_loss)

Test accuracy: 0.6111111111111112
Test loss: 1.3367948465877109


In [ ]:
print("test examples:", len(test_dl.dataset))


test examples: 36
